<a href="https://colab.research.google.com/github/Mar-vm/VitalIA/blob/backend/Vitalia1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# MONTAR DRIVE
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# PREPARACION DE LOS DATOS

import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



In [ ]:
# Ruta del CSV y carpeta con imágenes
csv_path = '/content/drive/MyDrive/HAM10000/HAM10000_metadata.csv'
img_folder = '/content/drive/MyDrive/HAM10000/'



In [ ]:
# Leer CSV
df = pd.read_csv(csv_path)

In [ ]:

# Añadir extensión .jpg al nombre de imagen
df['image_path'] = df['image_id'].apply(lambda x: os.path.join(img_folder, x + '.jpg'))

In [ ]:
df = df[df['image_path'].apply(os.path.exists)]

KeyError: 'image_path'

In [ ]:
# Codificar etiquetas en números
le = LabelEncoder()
df['label'] = le.fit_transform(df['dx'])

KeyError: 'dx'

In [ ]:
# Ver clases
print("Etiquetas codificadas:", dict(zip(le.classes_, le.transform(le.classes_))))

Etiquetas codificadas: {}


Preprocesar imagenes para dividirlas en conjuntos

In [ ]:
print(df.head())  # Verifica las primeras filas del DataFrame
print(len(df))    # Verifica cuántas filas tiene el DataFrame


Empty DataFrame
Columns: [lesion_id, image_id, dx, dx_type, age, sex, localization, image_path, label]
Index: []
0


In [ ]:
import numpy as np
from PIL import Image
from tqdm import tqdm

# Tamaño estándar de imágenes
IMG_SIZE = 64  # Puedes usar 128 o 224 si tienes recursos

# Cargar imágenes y etiquetas
X = []
y = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    img = Image.open(row['image_path']).resize((IMG_SIZE, IMG_SIZE))
    X.append(np.array(img))
    y.append(row['label'])

X = np.array(X) / 255.0  # Normalizar
y = np.array(y)

# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)



0it [00:00, ?it/s]


ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

Crear modelo CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(le.classes_), activation='softmax')  # 7 clases
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Training the model

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)


NameError: name 'model' is not defined

Save the model

In [ ]:
model.save('/content/drive/MyDrive/HAM10000/skin_model_v1.h5')

NameError: name 'model' is not defined